In [1]:
import torch
import torchvision
from torchvision import models
import torchvision.transforms as T
import PIL
from PIL import Image
import numpy as np

In [2]:
# 4 cpu to use
torch.set_num_threads(4)

In [3]:
#fasterrcnn_resnet50_fpn
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [4]:
def my_get_prediction(img_path, threshold):
    #prepare
    img = Image.open(img_path)
    transform = T.Compose([T.Resize(size=(224,224)), T.ToTensor()])
    img = transform(img)
    if img.size()[0] > 3:
        img = img[0:3]
    #inference
    pred = model([img])
    pred_score = list(pred[0]['scores'].detach().numpy())
    pred_class_num = list(pred[0]['labels'].detach().numpy())
    idx_res = [pred_score.index(x) for x in pred_score if x>threshold]

    ### empty output
    if idx_res.count(0) == 0:
        pred_score = [1]
        pred_class_num = [999]
    else:
        pred_class_num = pred_class_num[:len(idx_res)]
        pred_score = pred_score[:len(idx_res)]
    return pred_class_num, pred_score

def my_get_prediction_person(img_path, threshold, threshold_pers):
    pred_class_num, pred_score = my_get_prediction(img_path, threshold)
    output = []
    # unique class
    for class_num in pred_class_num:
        if class_num not in output:
            output.append(class_num)
    ### person
    person = 0
    indx_list = []
    indx_list = [i for i, x in enumerate(pred_class_num) if x == 1] 
    for indx in indx_list:
        if pred_score[indx] > threshold_pers:
            person += 1
    
    ### unique class with treshold < 5 and person is ok
    if (len(output) < 5) and (person > 0) :
        prediction = 1
    else:
        prediction = 0
        
    return prediction



### To prod use my_get_prediction_person with threshold=0.2 , threshold_pers = 0.95

### Some tests

In [15]:
#%%time
my_get_prediction_person('./1492252053_w640_h640_kukla-ken-barbi.jpg', threshold=0.2, threshold_pers = 0.95)
#my_get_prediction_person('./яхта.jpg', threshold=0.2, threshold_pers = 0.95)

1